In [ ]:
#%matplotlib inline
import os 
import argparse
import random
import google_BtoA
random.seed()
os.environ["THEANO_FLAGS"]  = "mode=FAST_RUN,device=gpu1,floatX=float32,lib.cnmem=1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
parser = argparse.ArgumentParser(description='Train model')                                                 
parser.add_argument('--backend', type=str, default="tensorflow", help="theano or tensorflow")                      
parser.add_argument('--generator', type=str, default="upsampling", help="upsampling or deconv or subpixel")    
parser.add_argument('--discriminator', type=str, default="discriminator", help="discriminator discriminator_resnet")
parser.add_argument('--dset', type=str, default="mnist", help="mnistM or washington_vandal50k or washington_vandal12classes")             
parser.add_argument('--img_dim', default=64, type=int, help="Image width == height")                           
parser.add_argument('--nb_epoch', default=10000, type=int, help="Number of epochs")                              
parser.add_argument('--batch_size', default=32, type=int, help='Batch size for training purpose; for testing is hardcoded at 32')                                   
parser.add_argument('--n_batch_per_epoch', default=2000, type=int, help="Number of batch per epochs")           
parser.add_argument('--bn_mode', default=2, type=int, help="Batch norm mode")                                  
parser.add_argument('--noise_dim', default=100, type=int, help="noise sampler dimension")                      
parser.add_argument('--noise_scale', default=0.5, type=float, help="noise sampler variance")                   
parser.add_argument('--disc_iterations', default=5, type=int, help="Number of discriminator iterations")         
parser.add_argument('--opt_D', type=str, default="Adam", help="Optimizer for the discriminator")            
parser.add_argument('--opt_G', type=str, default="Adam", help="Optimizer for the generator")                
parser.add_argument('--lr_D', type=float, default=1E-4, help="learning rate for the discriminator")            
parser.add_argument('--lr_G', type=float, default=1-4, help="learning rate for the generator")                
parser.add_argument('--use_mbd', action="store_true", help="use mini batch disc")
parser.add_argument('--deterministic', action="store_true", help="remove the noise input")
parser.add_argument('--inject_noise', action="store_true", help="inject noise into each layer")
parser.add_argument('--model', type=str, default="lsgan", help="wgan or lsgan") 
parser.add_argument('--no_supertrain', action="store_true", help="no discriminator supertraining") 
parser.add_argument('--pureGAN', action="store_true", help="no discriminator supertraining")
parser.add_argument('--lsmooth', type=float, default=1.0, help="label smoothing")
parser.add_argument('--monsterClass', action="store_true", help="Discriminator with 2x classes")
parser.add_argument('--disc_type', type=str, default="simple_disc", help="Discriminator type,between simple_disc or nclass_disc")
parser.add_argument('--resume', action="store_true", help="load GAN weights from previous train")#lsgan_VandWash1 is the name for the run Vand Wash no deterministic, no monsterclass
parser.add_argument('--name', type=str, default="MnistMtoMnist_dataAug", help="name of the run, used for saving and loading weights")
parser.add_argument('--wd', type=float, default=0.0, help="weight decay for generator/discriminator")
parser.add_argument('--history_size', type=float, default=1.8, help="history pool size = history_size * batch_size")
parser.add_argument('--data_aug', action="store_true", help="perform data augmentation on target dataset")

args = parser.parse_args(' --dset MnistMtoMnist --model lsgan --data_aug --resume --history_size 1.5 --no_supertrain --disc_iterations 1 --wd 1E-5 --lr_D 1E-4 --lr_G 1E-4'.split())
######7
print args
assert args.dset in ["mnistM","washington_vandal50k","washington_vandal12classes","washington_vandal12classesNoBackground",
                     "Wash_Vand_12class_LMDB","OfficeDslrToAmazon","bedrooms","Vand_Vand_12class_LMDB","MnistMtoMnist"]                                                      
assert args.opt_G in ["RMSprop", "SGD", "Adam", "AdamWithWeightnorm","SGDWithWeightnorm"], "Unsupported optimizer"                 
assert args.opt_D in ["RMSprop", "SGD", "Adam", "AdamWithWeightnorm","SGDWithWeightnorm"], "Unsupported optimizer" 

# Set the backend by modifying the env variable                                                                
if args.backend == "theano":                                                                                   
    os.environ["KERAS_BACKEND"] = "theano"                                                                     
elif args.backend == "tensorflow":                                                                             
    os.environ["KERAS_BACKEND"] = "tensorflow"  
import keras.backend as K
# manually set dim ordering otherwise it is not changed                                                        
#if args.backend == "theano":                                                                                   
#    image_dim_ordering = "th"                                                                                  
#    K.set_image_dim_ordering(image_dim_ordering)                                                               
#elif args.backend == "tensorflow":                                                                             
image_dim_ordering = "th"                                                                                  
K.set_image_dim_ordering(image_dim_ordering)

# Set default params
d_params = {"generator": args.generator,
            "discriminator": args.discriminator,
            "dset": args.dset,
            "img_dim": args.img_dim,
            "nb_epoch": args.nb_epoch,
            "batch_size": args.batch_size,
            "n_batch_per_epoch": args.n_batch_per_epoch,
            "bn_mode": args.bn_mode,
            "noise_dim": args.noise_dim,
            "noise_scale": args.noise_scale,
            "disc_iterations": args.disc_iterations,
            "lr_D": args.lr_D,
            "lr_G": args.lr_G,
            "opt_D": args.opt_D,
            "opt_G": args.opt_G,
            "use_mbd": args.use_mbd,
            "deterministic": args.deterministic,
            "pureGAN": args.pureGAN,
            "lsmooth": args.lsmooth,
            "image_dim_ordering": image_dim_ordering,
            "model": args.model,
            "no_supertrain": args.no_supertrain,
            "monsterClass": args.monsterClass,
            "data_aug": args.data_aug,
            "disc_type": args.disc_type,
            "resume": args.resume,
            "name": args.name,
            "inject_noise": args.inject_noise,
            "history_size" : args.history_size,
            "wd" : args.wd
            }

# Launch training
google_BtoA.train(**d_params)